In [ ]:
pip install beautifulsoup4

In [ ]:
pip install requests

In [ ]:
pip install lxml

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
## obtenemos el contenido estructural(en texto plano) de la página cambista
page = requests.get('https://www.infodolar.com.pe/')

## hacemos que ese contenido tenga formato HTML con BeautifulSoup
soup = BeautifulSoup(page.content, "html.parser")

In [5]:
table=soup.find('table', id='DolarPromedio')

In [6]:
table

<table class="cotizaciones" id="DolarPromedio">
<thead>
<tr>
<th class="dolar"></th>
<th class="dolar">Compra</th>
<th class="dolar">Venta</th>
<th class="dolar">Variación</th>
<th class="dolar">Spread</th>
<th class="dolar"><i class="far fa-clock"></i></th>
</tr>
</thead>
<tr>
<td class="colNombre">
<span class="nombre">Promedio InfoDolar</span>
</td>
<td class="colCompraVenta" data-order="3.7475">
                                S/3.748
                                <span class="variacionCompraVenta"><span class="alza"><i class="fa fa-arrow-up"></i> S/0.023</span></span>
</td>
<td class="colCompraVenta" data-order="3.77585">
                                S/3.776
                                <span class="variacionCompraVenta"><span class="alza"><i class="fa fa-arrow-up"></i> S/0.026</span></span>
</td>
<td class="colVariacion">
<span class="variacion"><span class="alza"><i class="fa fa-arrow-up"></i> 0.69 %</span></span>
</td>
<td class="colSpread">
                            

In [7]:
pd.read_html(str(table))[0]

/var/folders/qm/d416xgr55slc_jkcx9xkv1hh0000gn/T/ipykernel_92176/2644759292.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(table))[0]


,Unnamed: 0,Compra,Venta,Variación,Spread,Unnamed: 5
0,Promedio InfoDolar,S/3.748 S/0.023,S/3.776 S/0.026,0.69 %,S/0.028,"martes, 23 de enero de 2024 18:50 Perú"
1,Tipo de cambio SUNAT,S/3.736 = S/0.000,S/3.746 S/0.003,0.08 %,S/0.010,"martes, 23 de enero de 2024 09:10 Perú"


In [8]:
dolar_table = pd.read_html(str(table))[0]

/var/folders/qm/d416xgr55slc_jkcx9xkv1hh0000gn/T/ipykernel_92176/2838227840.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dolar_table = pd.read_html(str(table))[0]


In [9]:
dolar_table

,Unnamed: 0,Compra,Venta,Variación,Spread,Unnamed: 5
0,Promedio InfoDolar,S/3.748 S/0.023,S/3.776 S/0.026,0.69 %,S/0.028,"martes, 23 de enero de 2024 18:50 Perú"
1,Tipo de cambio SUNAT,S/3.736 = S/0.000,S/3.746 S/0.003,0.08 %,S/0.010,"martes, 23 de enero de 2024 09:10 Perú"


In [10]:
# dolar_table.loc[0,"Compra"] # Es lo mismo
list(dolar_table["Venta"])[0]

'S/3.776  S/0.026'

In [11]:
precio_dolar_sunat=list(dolar_table["Venta"])[0]
precio_dolar_sunat

'S/3.776  S/0.026'

In [12]:
precio_dolar_sunat.split(" ")[0]
precio_dolar_sunat.split(" ")[0].replace("S/","")
float(precio_dolar_sunat.split(" ")[0].replace("S/",""))

3.776

In [13]:
precio_dolar=float(precio_dolar_sunat.split(" ")[0].replace("S/",""))
precio_dolar

3.776

In [14]:
import sqlalchemy as db

In [ ]:
pip install sqlalchemy

In [ ]:
pip install mysql-connector-python

In [15]:
# este es el fichero para conectarnos a MySQL
class ConexionMySQL():
    def __init__(self, bd_servidor,bd_nombre,bd_usuario,bd_contrasenia):
        self.bd_servidor=bd_servidor
        self.bd_nombre=bd_nombre
        self.bd_usuario=bd_usuario
        self.bd_contrasenia=bd_contrasenia

    ## Función de conexión a MySQL
    def conectarMySQL(self):
        engine = db.create_engine(f"mysql+mysqlconnector://{self.bd_usuario}:{self.bd_contrasenia}@{self.bd_servidor}:3306/{self.bd_nombre}")
        
        conn=engine.connect()
        return conn

In [16]:
import json

In [ ]:
{
    "server":"xx.xx.xx.xx",
    "bd":"taller",
    "user":"root",
    "password":"password"
}

In [17]:
sql_path = "../local/credentials_mysql.json"

with open(sql_path, 'r') as archivo:
    credenciales_mysql = json.load(archivo)

In [18]:
## creando el objeto de la clase ConnetionMysql
obj_ventas = ConexionMySQL(
                            credenciales_mysql["server"],           ## Servidor
                            credenciales_mysql["bd"],               ## base de datos
                            credenciales_mysql["user"],             ## usuario
                            credenciales_mysql["password"]          ## contraseña
                            )

## llamar a la funcion de conexión a la base de datos
conn_ventas = obj_ventas.conectarMySQL()

In [19]:
import pandas as pd

In [20]:
query = '''
    select
        A.Factura as factura,
        A.Fecha as fecha,
        A.CodigoOperador as codigo_operador,
        B.RazonSocialOperador as operador,
        A.VentaTotal as venta_total
    from 
        ventas A
    left join
        operador B
    on A.CodigoOperador = B.CodigoOperador
'''

In [21]:
df_ventas=pd.read_sql(query, con=conn_ventas)

In [22]:
df_ventas["venta_total_usd"] = df_ventas["venta_total"]/precio_dolar

In [23]:
df_ventas["factura"]=df_ventas["factura"].astype(str)
df_ventas["codigo_operador"]=df_ventas["codigo_operador"].astype(str)

In [24]:
df_ventas.head()

,factura,fecha,codigo_operador,operador,venta_total,venta_total_usd
0,15,22/5/2014,1,Movistar,5238,1387.182203
1,326,30/8/2015,1,Movistar,1896,502.118644
2,364,30/10/2015,1,Movistar,3656,968.220339
3,680,12/2/2017,3,Bitel,5445,1442.002119
4,888,11/12/2017,3,Bitel,5971,1581.302966


In [ ]:
#df_ventas.rename(columns={"Factura":"factura","Fecha":"fecha","CodigoOperador":"codigo_operador","RazonSocialOperador":"operador"}, inplace=True)

In [ ]:
pip install google-cloud-bigquery

In [ ]:
# Para la carga a bigquery desde dataframe
pip install pyarrow

In [25]:
from google.cloud import bigquery

key_path = "../local/credentials_gcp.json"
client = bigquery.Client.from_service_account_json(key_path)

In [26]:
## id de la tabla nueva - cambiar el nombre al tuyo
table_id="taller-etl-datapath.taller_enero2024.ventas"

# creamos la estructura de la tabla
schema = []
table = bigquery.Table(table_id, schema=schema)

## enviando la instruccion a bigquery
table = client.create_table(table)

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/taller-etl-datapath/datasets/taller_enero2024/tables?prettyPrint=false: Already Exists: Table taller-etl-datapath:taller_enero2024.ventas

In [27]:
job_config = bigquery.LoadJobConfig(
    # especificamos el esquema de nuestra tabla
    schema=[
 # Especifique el tipo de columnas cuyo tipo no se puede detectar automáticamente.
        bigquery.SchemaField("factura", "STRING"),
        bigquery.SchemaField("fecha", "STRING"),
        bigquery.SchemaField("codigo_operador", "STRING"),
        bigquery.SchemaField("operador", "STRING"),
        bigquery.SchemaField("venta_total", "FLOAT"),
        bigquery.SchemaField("venta_total_usd", "FLOAT")
    ],

    write_disposition="WRITE_TRUNCATE", #WRITE_APPEND para sobrescribir
)

## proceso de ingesta
job = client.load_table_from_dataframe(
    df_ventas, # dataframe
    table_id, # tabla en bigquery
    job_config=job_config # configuación de carga
)

In [28]:
job.result()

LoadJob<project=taller-etl-datapath, location=US, id=efedd7fd-8bb1-4991-abad-20620d2fbf27>